In [1]:
# Import the necessary dependencies

# Operating System
import os

# Numpy, Pandas and Scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, save_npz, load_npz

# Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Model Evaluation
from evaluation import evaluate

# BLU12 - Learning Notebook - Workflow
## Introduction
This week we are going to simulate the real-life environment of the Hackathon! We will provide you with a dataset, some tips and you are going to train, validate and predict with a Recommender System using all the knowledge acquired throughout this specialization.

## Context
You have finished the Academy and you are hired as a Data Scientist in Recommentunify *(ignore the horrible company name, it works and we have more important stuff to discuss)*.

First week on the job and the CEO of the next-unicorn-startup that hired you pitches your first task for the company. NASA has discovered that "Despacito 2" is on the making and you need to **urgently avoid this scenario.**

<img src="./media/despacito_nasa.jpg" alt="Miss me yet?" width="500"/>

He doesn't care how you do it or how it works, just that you use fancy trendy eye-catching mind-blowing AI *stuff* to promote Recommentunify's name in the Data Science industry.

## Technical Task
After the pitch from the CEO, you talk with your Lead Data Scientist and you get the tasks needed to fulfill the job. The company collected some data on the users, you will need to create the Recommender System that is effective enough so that your users will not need a 2nd version of a Despacito.

Your Lead DS keeps some of the data as a test set (in which you will estimate production performance) and gives you the remaining data for yourself.

## Step -1: Understanding the data
You have available under the `./data/` folder some files that you can use for building the Recommender System. As a good scientist, the first step is to validate that the data you have will be good enough to proceed.

The files we have available for the **training** stage are:
* `train_play_counts.txt` has the listening history for each user
* `song_tag.csv`: has the relationship between tags and songs
* `songs.txt`: has the correspondence between the song_id (a unique id, eg: SOAAADD12AB018A9DD) and the song_index (an integer index, eg: 1)
* `tags.csv`: has the correspondence between the tag_id (a tag name, eg: classic rock) and the tag_index (an integer index, eg: 1)



For the **test** stage, we have:
* `test_users.csv`: has the user_ids for which a recommendation should be produced
* `example_output.csv`: has an example of an output (just for a couple of users)

The best approach is to look at the raw files and print out the first rows of each file just to get a overall sense of what we have.

*Note*: since the `song_tag.csv` is heavy, it was stored in a zip file which you will need to unzip first.

*Note for Windows users: remember that you need to convert the head command into the Windows equivalent. Tip: you can use what you learned in Data Wrangling Learning Unit!*

In [2]:
print("train_play_counts.txt \n")
!head -3 data/train_play_counts.txt

train_play_counts.txt 






In [3]:
#Unzip song_tag.zip first.
print("song_tag.csv\n")
!head -4 data/song_tag.csv

song_tag.csv

head: cannot open 'data/song_tag.csv' for reading: No such file or directory


In [4]:
print("songs.txt \n")
!head -3 data/songs.txt

songs.txt 






In [5]:
print("tags.csv \n")
!head -4 data/tags.csv

tags.csv 







In [6]:
print("test_users.csv \n")
!head -3 data/test_users.csv

test_users.csv 






In [7]:
print("example_output.csv \n")
!head -1 data/example_output.csv

example_output.csv 




## Step 0: Load the Data
After validating that the data we have is good enough, we start with building the ratings matrix. Our strategy for the first model is to get a non-personalized system as a baseline. This allows us to both predict for existing users (no cold-start problem) and for new users.

## Step 0.1: Load the Train and Test files

In [62]:
def read_users_history() -> pd.DataFrame:
    """Imports the listening history for each user.
    
    Returns:
        data (pd.DataFrame): DataFrame with the song and respective rating for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
    """
    path = os.path.join('data', 'train_play_counts.txt')
    data = pd.read_csv(path, names=['user', 'song_id', 'rating'], sep='\t')
    return data

data = read_users_history()
data.head()

,user,song_id,rating
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


In [63]:
# How many ratings do we have in total?
# Tip: The ":," at the end of the f-string adds the thousand separator.
print(f"We have {len(data):,} ratings in total.")

We have 100,000 ratings in total.


In [64]:
def read_test_users() -> pd.DataFrame:
    """Imports the list of users for which we need to predict.
    
    Returns:
        users_to_pred (pd.DataFrame): DataFrame with the users for which we will recommend songs. 
    """

    path = os.path.join('data', 'test_users.csv')
    users_to_pred_ = pd.read_csv(path, names=['users to recommend songs'])
    
    return users_to_pred_


users_to_pred = read_test_users()
users_to_pred.head()

,users to recommend songs
0,56d985c92960b98ad76a48b10a062b0cd86795bf
1,0102d1549242d159df98f333aee4041f96f37e98
2,991411f0dca94f348c7bd3eae93b6e6c061605f1
3,03feb8ee0424fc5c0bafb25e4bdc6d9380a3caba
4,323fbb28144eefa3eabfa22bd310dfb0713de80d


In [65]:
# For how many users are we recommending stuff?
print(f"We have {len(users_to_pred)} users in need for better songs!")

We have 1000 users in need for better songs!


## Step 0.2: Compare the train and test files

In [66]:
# And for how many users we already know something?
# For the 1000 users in our test set, we have data for 700 in the original listening history.
# For the remaining 300 we will have to use non-personalized strategies.
users_to_pred.isin(data.user.values).sum()

users to recommend songs    700
dtype: int64

In [68]:
def get_indices_from_users_to_pred(users_to_pred: pd.DataFrame, data: pd.DataFrame):
    """Get the indices of users_to_pred for which we have data and for which we don't.
    
    Args:
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        data (pd.DataFrame): Original of listening history for the users.
        
    Returns:
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
        index_users_not_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's without training data.
        
    """
    index_users_in_data = users_to_pred[users_to_pred.isin(data.user.values).values].index
    index_users_not_in_data = users_to_pred[~users_to_pred.isin(data.user.values).values].index
    
    return index_users_in_data, index_users_not_in_data


index_users_in_data, index_users_not_in_data = get_indices_from_users_to_pred(users_to_pred, data)

In [69]:
# For further inspection, we advise you to look at the objects themselves.
print(f"The index for users which we have training data has length of {len(index_users_in_data)}.")
print(f"The index for users which we don't have training data has length of {len(index_users_not_in_data)}.")

The index for users which we have training data has length of 700.
The index for users which we don't have training data has length of 300.


In [70]:
def get_users_to_pred_by_index(users_to_pred: pd.DataFrame, index_users_in_data: pd.Int64Index):
    """DataFrame with user_id's in test set for for which we have training data.

    Args: 
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
    Returns:
        users_in_data (pd.DataFrame): Dataframe containing the list of user_id's with training data.
    
    """
    return users_to_pred.iloc[index_users_in_data].reset_index(drop=True)

# Get the test users with training data
test_users_in_data = get_users_to_pred_by_index(users_to_pred, index_users_in_data)
print(test_users_in_data.shape)
test_users_in_data.head()

(700, 1)


,users to recommend songs
0,56d985c92960b98ad76a48b10a062b0cd86795bf
1,991411f0dca94f348c7bd3eae93b6e6c061605f1
2,323fbb28144eefa3eabfa22bd310dfb0713de80d
3,55c750f0951ca1021b26c0e758660bb8a2c49d3a
4,b458e3d697276a93aa6926caf1ff08e875933940


In [71]:
# Get the test users without training data
test_users_not_in_data = get_users_to_pred_by_index(users_to_pred, index_users_not_in_data)
print(test_users_not_in_data.shape)
test_users_not_in_data.head()

(300, 1)


,users to recommend songs
0,0102d1549242d159df98f333aee4041f96f37e98
1,03feb8ee0424fc5c0bafb25e4bdc6d9380a3caba
2,02e4174f754ee4546f4c4438a4bea6ed877c6c4f
3,03070a3284db565b2e67f8ad01ce96c31c9986dd
4,01dd0b2bb39643b8925a672105819d599f827c87


## Step 0.3: Create the Ratings matrix

Here we are creating a rating matrix by instantiating the csr_matrix in a different way than before. Previously, a dense matrix (actually a `pd.DataFrame`) was created with the `.pivot()` method, the indices were ordered, the missing values were replaced with 0 and then the dataframe was used as argument of `csr_matrix`. This is an intensive task, time and memory wise. To avoid having to create a huge dataframe it is better to get the indices of users and items and the respective values (all `ndarray`s) and use the fourth instantiation method described in the [csr_matrix documentation]((https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)).

In [151]:
def make_ratings(data: pd.DataFrame, shape: tuple = None) -> csr_matrix:
    """Creates the ratings matrix of listening history with optional shape
    
    Creates the ratings matrix from the listening history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  Listening history for the users.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from data argument.
        
    Returns:
        ratings (csr_matrix): Ratings matrix with shape (n_users, n_items).
    
    """
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    #If shape is not defined, it is determined by the data.
    #If shape explicitly defined then that shape is used.
    if shape == None:
        shape = (len(users), len(items))

    R_ = csr_matrix((values, (user_pos, item_pos)), shape=shape)
    return R_


R = make_ratings(data)
R

<7526x41194 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [73]:
# Just for mental (in)sanity, let's match the info of the matrix to what is printed in the previous cell.
print(f"The shape is {R.shape}")
print(f"The dtypes of the elements are {R.dtype}")
print(f"The number of stored elements is {R.nnz}")
print(f"The type of the matrix is {type(R)}")

The shape is (7526, 41194)
The dtypes of the elements are int64
The number of stored elements is 100000
The type of the matrix is <class 'scipy.sparse.csr.csr_matrix'>


In [74]:
# Let's store a Series with the unique user id's that we have in the original data.
def get_unique_users(data: pd.DataFrame) -> pd.DataFrame:
    """Get unique users in training data.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        unique_users (pd.DataFrame): DataFrame of one column with unique users in training data.
    
    """
    return pd.DataFrame(np.unique(data.iloc[:, 0].values), columns=["users to recommend songs"])


unique_users_training_data = get_unique_users(data)
unique_users_training_data.head()

,users to recommend songs
0,0011d5f4fb02ff276763d385c3f2ded2b00ad94a
1,002511b392561fc1d426d875c386b356a6fc5702
2,002dfbc3c073b55a64a4abab34c0ca1f13897f1c
3,003998bc33cddeba02428a43391c6716e523c8f7
4,0042d2027dfa0340e31d2aa875c4be229730efb7


## Step 0.4: Record the indices of users
After having the users for the test set and the original ratings matrix, let's create some indices which we will use later on to filter data. Don't worry if it does not seem much intuitive in the beginning, you will further return to this and get a better grasp.

In [75]:
def get_indices_in_ratings_for_test_users_in_data(data: pd.DataFrame, test_users_in_data: pd.DataFrame) -> np.array:
    """Returns the index of the ratings matrix for the test users for which we have training data.
    
    Args:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
        test_users_in_data (pd.DataFrame): DataFrame containing the list of test users for which we have training data.
        
    Returns:
        indices_ratings_tests_users_in_data (np.array): Indices of users in test set with training data for ratings matrix.
    
    """
    unique_users = get_unique_users(data).iloc[:, 0]
    indices_ratings_tests_users_in_data = unique_users[unique_users.isin(test_users_in_data.iloc[:, 0])].index.to_numpy()                                                                                       
    return indices_ratings_tests_users_in_data

indices_ratings_tests_users_in_data = get_indices_in_ratings_for_test_users_in_data(data, test_users_in_data)

In [76]:
len(indices_ratings_tests_users_in_data)

700

In [77]:
print(f"As expected, the length of the indices should be {len(indices_ratings_tests_users_in_data)}, matching the number of users in test set with training data.")

As expected, the length of the indices should be 700, matching the number of users in test set with training data.


## Step 1: Understand your data flow
In the end, we will be predicting over a test set of users, which may or may not have ratings available in our training data. For the ones in the test set for which we have data in the training, we may use personalized recommendation systems. Otherwise, we need to switch to non-personalized recommendations. 

To split into train and validation, we undersample the original list of ratings but we need to make sure that the overall shape is the same (making sure that by undersampling we do not affect the shape of the full matrix).  This is, the universe of items and users is still the same. We simply have less ratings to work with.

In [135]:
# Percentage of listening history used for validation.
test_size = 0.2

def make_train_val_split(data: pd.DataFrame, test_size : float = 0.2, shape: tuple = None):
    """Split the data into train and validation and returns the ratings matrixes accordingly.
    
    Args:
        data (pd.DataFrame): Listening history for the users.
        test_size (float): Percentage of listening history used for validation.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from 'data' argument.
    
    Returns:
        ratings_train (csr_matrix): Ratings matrix for train.
        ratings_val (csr_matrix): Ratings matrix for validation.
    
    """
    train_data, val_data = train_test_split(data, test_size=test_size, random_state=8)

    #Store the indexes of each observation
    train_index = train_data.index
    val_index = val_data.index

    #make copies of data to replace the observations
    train_data_clean = data.copy()
    val_data_clean = data.copy()

    #Replace the validation observations on the training data
    train_data_clean.loc[val_index,["rating"]] = 0
    
    #Replace the training observations on the validation data
    val_data_clean.loc[train_index,["rating"]] = 0

    R_train = make_ratings(train_data_clean, shape=shape)
    R_val = make_ratings(val_data_clean, shape=shape)

    #remove the explicit zeros from the sparse matrices
    R_train.eliminate_zeros()
    R_val.eliminate_zeros()

    return R_train, R_val

ratings_train, ratings_val = make_train_val_split(data, test_size=test_size, shape=R.shape)

In [136]:
# After the train/validation split, let's compare the number of ratings available in each matrix.
print(f"After the split we have {ratings_train.nnz:,} ratings in the train set and {ratings_val.nnz:,} ratings in the validation set.")

After the split we have 80,000 ratings in the train set and 20,000 ratings in the validation set.


## Step 2: Non-Personalized
Let's build our baseline using one of the the Non-Personalized techniques we learned in BLU10 - "Popular Items". This basically fetches the items which have more ratings overall (we don't care if it is good or bad). Is this how it should be?

<img src="./media/fry_good_bad.jpg" alt="Confused Fry" width="300"/>


Probably not. But it is worth a shot!

In [137]:
def get_most_rated(ratings: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items in a ratings matrix.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    def is_rating(R_: csr_matrix) -> csr_matrix:
        """Returns a sparse matrix of booleans 
        
        Args:
            R_ (csr_matrix): A sparse ratings matrix
            
        Returns:
            is_rating (csr_matrix): A sparse matrix of booleans.
        """
        return R_ > 0
    
    def count_ratings(R_: csr_matrix) -> np.matrix:
        """Returns an array with the count of ratings
        
        The attribute ".A1" of a numpy matrix returns self as a flattened ndarray.
        
        Args:
            R_ (csr_matrix): A sparse matrix of booleans
        
        Returns:
            count_ratings (np.darray): Count of ratings by item
        """
        return R_.sum(axis=0).A1
    
    ratings_ = is_rating(ratings)
    ratings_ = count_ratings(ratings_)
    return np.negative(ratings_).argsort()[:n]


non_pers_most_rated = get_most_rated(ratings_train, 100)
non_pers_most_rated

array([ 9622,  1394,  2648,  1557, 30752, 23122,  7282, 18742,  5794,
       33598,  9217, 33416, 32150, 25953, 25737, 26012, 13142, 35433,
        2667, 17537,  5588,  4849, 34014, 23589, 32656, 19467, 37102,
        9121,  2616, 29486,  4153, 32168, 30860, 33143, 18055, 26211,
       35544, 35539,   221, 11151, 19105, 33009, 22051, 35892, 22119,
       35069, 32651, 25916, 31027, 22055, 25784,   930, 19464,  5555,
       24856, 22485, 20368, 19794, 35690, 22428, 14716, 32537, 37772,
       38099, 36319, 40907, 18788, 37604, 31121, 13480, 22565, 28416,
       37667, 30413, 10324, 18776, 37450, 32111, 38643,  1739,  9581,
       33098,  6983, 15124, 29939, 11471,  2062, 26777,  2707, 39865,
       36734, 26126, 17570, 20094, 40950,  5286, 11184, 29148, 23803,
        7891])

In [138]:
def convert_non_pers_recommendations_to_df(non_pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the non-personalized most rated to an DataFrame with the users and the recommendations.
    We will basically repeat the non_pers_recs array for the number of users in need.
    
    Args:
        non_pers_recs (np.array): Array of indices for the best non-personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    non_pers_df = pd.DataFrame(np.zeros((len(users_to_pred), 1), dtype=non_pers_recs.dtype) + non_pers_recs)
    non_pers_df = pd.concat([users_to_pred, non_pers_df], axis=1)
    non_pers_df = non_pers_df.set_index("users to recommend songs")
    
    return non_pers_df


non_pers_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, unique_users_training_data)
non_pers_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,9622,1394,2648,1557,30752,23122,7282,18742,5794,33598,...,36734,26126,17570,20094,40950,5286,11184,29148,23803,7891
002511b392561fc1d426d875c386b356a6fc5702,9622,1394,2648,1557,30752,23122,7282,18742,5794,33598,...,36734,26126,17570,20094,40950,5286,11184,29148,23803,7891
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,9622,1394,2648,1557,30752,23122,7282,18742,5794,33598,...,36734,26126,17570,20094,40950,5286,11184,29148,23803,7891
003998bc33cddeba02428a43391c6716e523c8f7,9622,1394,2648,1557,30752,23122,7282,18742,5794,33598,...,36734,26126,17570,20094,40950,5286,11184,29148,23803,7891
0042d2027dfa0340e31d2aa875c4be229730efb7,9622,1394,2648,1557,30752,23122,7282,18742,5794,33598,...,36734,26126,17570,20094,40950,5286,11184,29148,23803,7891


In [139]:
def create_dict_preds(preds_df: pd.DataFrame) -> dict:
    """Convert the predictions DataFrame (index:users -> columns: items) to a dictionary of key (user->list of items).
    
    Args: 
        preds_df (pd.DataFrame): DataFrame containing the users and the ordered predictions.
        
    Returns:
        preds_dict (dict): Dict of (user_id: list of items) used for evaluating the performance.
    
    """
    return {preds_df.index[i]: preds_df.values[i].tolist() for i in range(len(preds_df))}


non_pers_dict = create_dict_preds(non_pers_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(non_pers_dict.items())[0:1])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [9622,
  1394,
  2648,
  1557,
  30752,
  23122,
  7282,
  18742,
  5794,
  33598,
  9217,
  33416,
  32150,
  25953,
  25737,
  26012,
  13142,
  35433,
  2667,
  17537,
  5588,
  4849,
  34014,
  23589,
  32656,
  19467,
  37102,
  9121,
  2616,
  29486,
  4153,
  32168,
  30860,
  33143,
  18055,
  26211,
  35544,
  35539,
  221,
  11151,
  19105,
  33009,
  22051,
  35892,
  22119,
  35069,
  32651,
  25916,
  31027,
  22055,
  25784,
  930,
  19464,
  5555,
  24856,
  22485,
  20368,
  19794,
  35690,
  22428,
  14716,
  32537,
  37772,
  38099,
  36319,
  40907,
  18788,
  37604,
  31121,
  13480,
  22565,
  28416,
  37667,
  30413,
  10324,
  18776,
  37450,
  32111,
  38643,
  1739,
  9581,
  33098,
  6983,
  15124,
  29939,
  11471,
  2062,
  26777,
  2707,
  39865,
  36734,
  26126,
  17570,
  20094,
  40950,
  5286,
  11184,
  29148,
  23803,
  7891]}

### Step 2.2 - Creating the Ground Truth
Since we are splitting the data into train and validation, we will test the predictions we get with the training data in the validation set. To do this, we need to determine what is the ground truth (the actual outcomes) of the recommendations for the validation set. 

The main idea is to pick up a sorted array for each user on the songs with highest value in the validaton ratings matrix.

In [140]:
unique_users_training_data

,users to recommend songs
0,0011d5f4fb02ff276763d385c3f2ded2b00ad94a
1,002511b392561fc1d426d875c386b356a6fc5702
2,002dfbc3c073b55a64a4abab34c0ca1f13897f1c
3,003998bc33cddeba02428a43391c6716e523c8f7
4,0042d2027dfa0340e31d2aa875c4be229730efb7
...,...
7521,ffd41f0f4c56e011d86a5005439f3468fd29d1d9
7522,ffdfbc60afdcdcb630d3b667ca3a083b09ed6212
7523,ffdfc7f9864ee172c3488707969c90e7b1ac4dc7
7524,ffe2811be1a471ea1d30fd646d815d272aef7d4d


In [141]:
def get_y_true(R_val_: csr_matrix, users_to_pred: pd.DataFrame, n=100):
    """Get the ground truth (best recommendations) of the users in the validation set.
    
    Args:
        R_val_ (csr_matrix): Validation set ratings matrix.
        users_to_pred: 
        n (int): Number of top-n items.
        
    Returns:
        y_true_df (pd.DataFrame): DataFrame which returns the y_true items.
        
    """
    top_from_R_val = pd.DataFrame(np.negative(R_val_).toarray().argsort()[:, :n])
    y_true_df = pd.concat([users_to_pred, top_from_R_val], axis=1)
    y_true_df = y_true_df.set_index("users to recommend songs")
    return y_true_df


y_true_df = get_y_true(ratings_val, unique_users_training_data, n=100)
y_true_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,38805,15053,1447,11133,18776,27461,27462,27463,27464,27465,...,27387,27389,27390,27391,27392,27393,27394,27395,27388,27396
002511b392561fc1d426d875c386b356a6fc5702,17243,37411,27458,27459,27460,27461,27462,27463,27457,27464,...,27384,27386,27387,27388,27389,27390,27391,27392,27385,27393
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,37456,1003,2208,30222,1945,3975,27461,27462,27463,27464,...,27385,27386,27379,27387,27389,27390,27391,27392,27393,27394
003998bc33cddeba02428a43391c6716e523c8f7,38260,20340,38279,0,27457,27458,27459,27460,27461,27462,...,27385,27386,27387,27388,27389,27390,27391,27392,27377,27376
0042d2027dfa0340e31d2aa875c4be229730efb7,0,27457,27458,27459,27460,27461,27462,27463,27456,27464,...,27386,27387,27388,27389,27390,27391,27392,27385,27393,27375


In [142]:
# Create the dictionary with the ground truth.
y_true_dict = create_dict_preds(y_true_df)

## Evaluate
We will use the Mean Average Precision @ K to evaluate our predictions.

In [143]:
evaluate(y_true_dict, non_pers_dict)

0.0003010999155361327

## Predict 
To submit your predictions, you just need to convert your recommendations to the format we have in the `example_output.csv` file.

In [32]:
# Join both dataframes with user_id's
all_test_users = pd.concat([test_users_in_data, test_users_not_in_data]).reset_index(drop=True)

In [33]:
non_pers_test_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, all_test_users)
print(non_pers_test_most_rated_df.shape)
non_pers_test_most_rated_df.head()

(1000, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
991411f0dca94f348c7bd3eae93b6e6c061605f1,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
323fbb28144eefa3eabfa22bd310dfb0713de80d,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
55c750f0951ca1021b26c0e758660bb8a2c49d3a,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
b458e3d697276a93aa6926caf1ff08e875933940,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409


In [34]:
def save_predictions(predictions: pd.DataFrame, output_path: str):
    """Save predictions to csv.
    
    Saves the predictions into a csv file with the format we need.
    We keep the index since it contains the user ids.
    
    Args:
        predictions (pd.DataFrame): DataFrame with user_id as index and ordered recommendations in the columns.
        output_path (str): Filepath for the predictions file.
    
    """
    predictions.to_csv(output_path, header=None)
    print(f"Saved to csv in '{output_path}'.")
    
    
save_predictions(non_pers_test_most_rated_df, os.path.join("data", "test_non_personalized_recommendations.csv"))

Saved to csv in 'data/test_non_personalized_recommendations.csv'.


In [35]:
# Filter the non-personalized recommendations for the users without training data and save
new_users_recommendations = non_pers_most_rated_df.iloc[index_users_not_in_data]
save_predictions(new_users_recommendations, os.path.join("data", "new_users_non_personalized.csv"))

Saved to csv in 'data/new_users_non_personalized.csv'.


## Step 3: Personalized


### Step 3.1 - Collaborative filtering


In [144]:
def make_user_similarities(R_: csr_matrix) -> csr_matrix:
    """Creates the user similarities matrix.
    
    Args:
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        user_similarities (csr_matrix): Matrix with user similarities.
    
    """
    return cosine_similarity(R_, dense_output=False)


user_similarities = make_user_similarities(ratings_train)
user_similarities

<7526x7526 sparse matrix of type '<class 'numpy.float64'>'
	with 1078541 stored elements in Compressed Sparse Row format>

In [145]:
def make_user_predictions_collab_filt(S: csr_matrix, R_: csr_matrix):
    """Predict using collaborative filtering.
    
    Args:
        S (csr_matrix): Similarities matrix (tipically using the cosine_similarity).
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        preds (csr_matrix): Predictions matrix.
    
    """
    weighted_sum = np.dot(S, R_)
    
    # We use the absolute value to support negative similarities.
    # In this particular example there are none.
    sum_of_weights = np.abs(S).sum(axis=1)
    
    preds = weighted_sum / sum_of_weights
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return csr_matrix(preds)
 

collab_filt_user_preds = make_user_predictions_collab_filt(user_similarities, ratings_train)
collab_filt_user_preds

<7526x41194 sparse matrix of type '<class 'numpy.float64'>'
	with 9269874 stored elements in Compressed Sparse Row format>

In [146]:
def sparsity(matrix: csr_matrix) -> float:
    """Calculates the sparsity of a matrix.
    
    Args:
        matrix (csr_matrix): Sparse matrix.
        
    Returns:
        sparsity_ (float): Sparsity percentage (between 0 and 1).
    
    """
    return 1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])


sparsity(collab_filt_user_preds)

0.9700996926567885

In [147]:
def get_most_rated_from_user_preds(user_preds_: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items from the user predictions.
    
    Args:
        user_preds_ (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    pred_ = np.negative(user_preds_).toarray()
    return pred_.argsort()[:, :n]


collab_filt_most_rated = get_most_rated_from_user_preds(collab_filt_user_preds, 100)
print(collab_filt_most_rated.shape)
collab_filt_most_rated

(7526, 100)


array([[36432, 29939,  2648, ..., 21687,  9121,  7691],
       [31901, 25323, 14131, ..., 22554, 40790, 33750],
       [ 9966, 34014, 31046, ...,  7085,  2377,  1201],
       ...,
       [12171,  2936,  1436, ..., 27480, 27454, 27482],
       [ 6234,  1557,  1394, ...,  7598, 15159, 29817],
       [31509, 35670, 16451, ..., 14524, 36116,  3070]])

In [148]:
def convert_pers_recommendations_to_df(pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """Converts the personalized most rated to an DataFrame with the users and the recommendations.
    
    Args:
        pers_recs (np.array): Array of indices for the best personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    pers_df = pd.concat([users_to_pred, pd.DataFrame(pers_recs)], axis=1)
    pers_df = pers_df.set_index("users to recommend songs")
    
    return pers_df


collab_filt_most_rated_df = convert_pers_recommendations_to_df(collab_filt_most_rated, unique_users_training_data)
collab_filt_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,36432,29939,2648,20368,38277,12890,33331,22565,35406,10324,...,22571,25953,11151,31756,32530,13733,4204,21687,9121,7691
002511b392561fc1d426d875c386b356a6fc5702,31901,25323,14131,34763,16020,38354,27305,5232,37434,9191,...,30647,3367,15182,26669,17383,28997,4761,22554,40790,33750
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,9966,34014,31046,9622,2208,7282,7598,23122,10269,33143,...,10324,21352,35592,26830,14134,23803,29486,7085,2377,1201
003998bc33cddeba02428a43391c6716e523c8f7,1612,1232,4276,33318,40377,25964,40950,991,26411,1658,...,38044,28931,15987,25147,37859,37842,8812,10654,5832,13737
0042d2027dfa0340e31d2aa875c4be229730efb7,4415,36171,29494,20028,31334,39976,33098,18742,35834,24048,...,2377,15114,18584,4840,12049,5188,24742,11238,30810,38725


In [149]:
collab_filt_dict = create_dict_preds(collab_filt_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(collab_filt_dict.items())[0:1])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [36432,
  29939,
  2648,
  20368,
  38277,
  12890,
  33331,
  22565,
  35406,
  10324,
  11658,
  33098,
  16428,
  9622,
  36319,
  30752,
  30933,
  5588,
  35539,
  32656,
  13363,
  13754,
  32684,
  33159,
  14705,
  33416,
  33110,
  5167,
  29015,
  40597,
  34495,
  17659,
  1394,
  11357,
  26445,
  9614,
  9675,
  33598,
  16044,
  17944,
  3592,
  13358,
  3669,
  6122,
  24229,
  34014,
  23944,
  34087,
  27882,
  14765,
  23589,
  26019,
  10352,
  29468,
  41019,
  24805,
  25916,
  39983,
  24965,
  33505,
  30860,
  18776,
  28267,
  7282,
  3288,
  15053,
  32537,
  1557,
  36116,
  34569,
  2616,
  25590,
  28926,
  25737,
  26261,
  24874,
  18506,
  29565,
  35967,
  23122,
  32168,
  38718,
  38805,
  15296,
  22051,
  28334,
  30247,
  23139,
  19105,
  14553,
  22571,
  25953,
  11151,
  31756,
  32530,
  13733,
  4204,
  21687,
  9121,
  7691]}

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [150]:
evaluate(y_true_dict, collab_filt_dict)

0.032606521925826186

## Predict 
To submit your predictions, you just need to convert your recommendations to the format we have in the `example_output.csv` file.

In [43]:
# Filter the collaborative filtering most rated DataFrame  using the test_users_in_data mask.
collab_filt_most_rated_in_data_df = collab_filt_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(collab_filt_most_rated_in_data_df.shape)
collab_filt_most_rated_in_data_df.head()

(700, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,1748,22253,621,27878,19066,5522,18590,6508,21687,31371,...,17858,3618,27507,27474,27479,27478,27509,27473,27508,27476
991411f0dca94f348c7bd3eae93b6e6c061605f1,25299,34096,32562,29865,19575,35181,22478,6494,30934,1228,...,8912,28751,9884,16319,3380,2154,8312,9469,14392,20438
323fbb28144eefa3eabfa22bd310dfb0713de80d,22234,2768,3310,4814,2563,34241,6994,21688,26562,5472,...,20255,3337,25810,5833,10057,16575,26550,13959,7045,35191
55c750f0951ca1021b26c0e758660bb8a2c49d3a,14423,1209,6418,18025,4850,16321,7195,16243,13629,26924,...,2537,4200,9844,25579,2252,19103,29113,1468,12054,1047
b458e3d697276a93aa6926caf1ff08e875933940,2290,8312,6308,28751,33175,29496,1778,30780,20048,1352,...,21868,11715,27449,35446,21772,24177,31942,970,3171,24364


In [44]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

(300, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
002511b392561fc1d426d875c386b356a6fc5702,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
003998bc33cddeba02428a43391c6716e523c8f7,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
005b1fab38cdeb9d5bb97debcf73b44050994a3e,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
005cc5d858319f13f88228f62341e5a4270f8e75,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
006d3c79b9ed677280f8ddbc422d7b0fedd6d1fa,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409


In [45]:
collab_filt_most_rated_test_df = pd.concat([collab_filt_most_rated_in_data_df, new_users_recommendations])
print(collab_filt_most_rated_test_df.shape)

(1000, 100)


In [46]:
# Save the collaborative filtering recommendations 
save_predictions(collab_filt_most_rated_test_df, os.path.join("data", "collab_filt_recommendations.csv"))

Saved to csv in 'data/collab_filt_recommendations.csv'.


### Step 3.2 - Content-based recommendations

In [82]:
# If you had a FileNotFoundError, make sure you have unziped the `song_tag.zip` file.
def read_tags() -> pd.DataFrame:
    """Import the song tags file.
    
    Returns:
        tags (pd.DataFrame): DataFrame with song_index, tag_index and value of the tag in the song.
        
    """
    path = os.path.join('data', 'song_tag.csv')
    data = pd.read_csv(path)
    return data


tags = read_tags()
print(tags.shape)
tags.head()

(4909471, 3)


,song_index,tag_index,val
0,254229,206,100.0
1,254229,1125,66.0
2,254229,582,66.0
3,254229,914,33.0
4,254229,95,33.0


In [83]:
def read_songs() -> pd.DataFrame:
    """Import the songs file.
    
    Returns:
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
    
    """
    path = os.path.join('data', 'songs.txt')
    data = pd.read_csv(path, names=['song_id', 'song_index'], sep=' ')
    return data


songs= read_songs()
print(songs.shape)
songs.head()

(386213, 2)


,song_id,song_index
0,SOAAADD12AB018A9DD,1
1,SOAAADE12A6D4F80CC,2
2,SOAAADF12A8C13DF62,3
3,SOAAADZ12A8C1334FB,4
4,SOAAAFI12A6D4F9C66,5


In [85]:
def merge_tags_songs(tags: pd.DataFrame, songs: pd.DataFrame, data: pd.DataFrame) -> pd.DataFrame:
    """Join the tags, songs and ratings data sources into a single dataframe).
    
    Args:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
        data (pd.DataFrame): Listening history for the users.

    Returns:
        tags_cleaned (pd.DataFrame): Matches the song_id to the tag_index with a corresponding value.
    
    """
    tags_cleaned = (tags.merge(songs, how='left', on='song_index')
                        .merge(data, how='right', on='song_id')
                   )[['song_id', 'tag_index', 'val']]
    return tags_cleaned

tags_cleaned = merge_tags_songs(tags, songs, data)
print(tags_cleaned.shape)
tags_cleaned.head()

(4773486, 3)


,song_id,tag_index,val
0,SOBONKR12A58A7A7E0,527.0,100.0
1,SOBONKR12A58A7A7E0,384.0,28.0
2,SOBONKR12A58A7A7E0,98.0,28.0
3,SOBONKR12A58A7A7E0,4070.0,14.0
4,SOBONKR12A58A7A7E0,139705.0,14.0


In [86]:
# Get the number of unique song_id's for which we have tags. 
tags_cleaned.song_id.nunique()

41194

In `make_item_profiles` below we are also instantiating `csr_matrix` using `ndarray`s instead of using `pd.DataFrame.pivot()`.

In [90]:
def make_item_profiles(data: pd.DataFrame) -> csr_matrix:
    """Creates the item profiles matrix.
    
    Args:
        data (pd.DataFrame): DataFrame containing the (rows, columns, values) for the ratings matrix.
                             In this case, we have (song_id, tag_index, value).
    
    Returns:
        item_profiles (csr_matrix): Item profiles matrix. Items as rows, tags as columns.
    
    """
    items, item_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    tags, tag_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(items), len(tags))

    item_profiles_ = csr_matrix((values, (item_pos, tag_pos)), shape=shape)
    return item_profiles_


item_profiles = make_item_profiles(tags_cleaned)
item_profiles

<41194x219644 sparse matrix of type '<class 'numpy.float64'>'
	with 1364190 stored elements in Compressed Sparse Row format>

In [92]:
def make_user_profiles(R_: csr_matrix, item_profiles_: csr_matrix) -> csr_matrix:
    """Calculate the user profiles with the items.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        
    Returns:
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
        
    """
    return np.dot(R_, item_profiles_)


user_profiles = make_user_profiles(ratings_train, item_profiles)
user_profiles

<7526x219644 sparse matrix of type '<class 'numpy.float64'>'
	with 1990023 stored elements in Compressed Sparse Row format>

In [93]:
def make_user_predictions_content(R_: csr_matrix, item_profiles_: csr_matrix, user_profiles_: csr_matrix) -> csr_matrix:
    """Produces content-based predictions.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
       
    Returns:
        preds (csr_matrix): Predictions of ratings using content-based recommendations.
    
    """
    
    # Since we are not looking at the values inbetween the cosine_similarity and excluding previously rated items,
    # we can define dense_output=False to speed up.
    preds = cosine_similarity(user_profiles_, item_profiles_, dense_output=False)
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return preds


content_user_preds = make_user_predictions_content(ratings_train, item_profiles, user_profiles)
content_user_preds

<7526x41194 sparse matrix of type '<class 'numpy.float64'>'
	with 196804067 stored elements in Compressed Sparse Row format>

In [94]:
content_most_rated = get_most_rated_from_user_preds(content_user_preds, 100)
print(content_most_rated.shape)
content_most_rated

(7526, 100)


array([[37668, 38513,  5656, ..., 16140, 17991, 15207],
       [23216, 27284,  2599, ..., 15536,  1020, 19335],
       [34570, 29126, 32327, ..., 33997, 35430, 22921],
       ...,
       [20603, 33715, 10530, ..., 23501, 37779, 34447],
       [21358, 39601,  2405, ..., 41188, 30743, 18640],
       [    0, 27457, 27458, ..., 27385, 27393, 27375]])

In [95]:
content_most_rated_df = convert_pers_recommendations_to_df(content_most_rated, unique_users_training_data)
print(content_most_rated_df.shape)
content_most_rated_df.head()

(7526, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,37668,38513,5656,16966,411,4285,20825,35242,22996,26463,...,10061,35081,19904,19394,23030,16154,32646,16140,17991,15207
002511b392561fc1d426d875c386b356a6fc5702,23216,27284,2599,24227,12260,32545,18295,16275,40523,15717,...,3378,32000,15879,37723,18830,32544,18281,15536,1020,19335
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,34570,29126,32327,33729,21206,7048,28306,30998,3676,14075,...,19569,36642,29298,29592,5035,10444,33949,33997,35430,22921
003998bc33cddeba02428a43391c6716e523c8f7,40865,23371,3422,12486,39739,40923,17608,24930,26236,37841,...,38476,13475,22242,11328,8356,3627,35935,10735,22181,22448
0042d2027dfa0340e31d2aa875c4be229730efb7,14192,25368,35300,5475,39809,32482,23056,16255,18012,39891,...,8250,31210,25192,9891,34373,17447,6565,255,24213,17735


In [96]:
content_dict = create_dict_preds(content_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(content_dict.items())[0:2])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [37668,
  38513,
  5656,
  16966,
  411,
  4285,
  20825,
  35242,
  22996,
  26463,
  3209,
  29770,
  25944,
  23080,
  35339,
  9427,
  28609,
  38650,
  30422,
  646,
  20268,
  14628,
  15670,
  21986,
  8082,
  6874,
  34464,
  37091,
  16429,
  21977,
  38240,
  30968,
  20069,
  8099,
  9482,
  23082,
  1023,
  31818,
  31094,
  32753,
  22275,
  18939,
  10769,
  35448,
  2392,
  33962,
  24648,
  37316,
  38657,
  25739,
  38509,
  29795,
  11615,
  17444,
  40984,
  33152,
  5859,
  8172,
  26637,
  3018,
  7676,
  17064,
  30701,
  28776,
  27009,
  34007,
  18801,
  40959,
  28849,
  11487,
  39481,
  15457,
  6068,
  32369,
  36022,
  12894,
  2895,
  20907,
  13714,
  13229,
  29218,
  15740,
  32239,
  20712,
  14868,
  22360,
  8393,
  41066,
  26153,
  786,
  10061,
  35081,
  19904,
  19394,
  23030,
  16154,
  32646,
  16140,
  17991,
  15207],
 '002511b392561fc1d426d875c386b356a6fc5702': [23216,
  27284,
  2599,
  24227,

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [97]:
evaluate(y_true_dict, content_dict)

0.0009937371360594822

## Predict 
To submit your predictions, you just need to convert your recommendations to the format we have in the `example_output.csv` file.

In [98]:
content_most_rated_in_data_df = content_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(content_most_rated_in_data_df.shape)
content_most_rated_in_data_df.head()

(700, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,34205,93,17641,15026,39627,25194,9051,35365,1814,2036,...,28445,10301,13567,11524,20549,25170,39312,16949,34879,29697
991411f0dca94f348c7bd3eae93b6e6c061605f1,14384,16672,7319,8682,26524,21879,33398,12991,709,8930,...,7497,23800,24572,35811,972,34469,28743,27197,279,36996
323fbb28144eefa3eabfa22bd310dfb0713de80d,26532,36900,40471,8476,31638,17357,15468,31757,2802,12893,...,23771,35357,23845,25988,18843,41007,406,31729,28073,6986
55c750f0951ca1021b26c0e758660bb8a2c49d3a,31759,10421,20366,39719,15188,13119,6235,12144,35116,38409,...,25063,29992,6916,20372,11439,34253,35966,1253,22540,2765
b458e3d697276a93aa6926caf1ff08e875933940,9406,10037,24926,6874,18815,25938,31243,31348,20375,33345,...,19298,35468,22720,27327,2535,40505,12961,4646,26924,7303


In [99]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

(300, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
002511b392561fc1d426d875c386b356a6fc5702,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
003998bc33cddeba02428a43391c6716e523c8f7,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
005b1fab38cdeb9d5bb97debcf73b44050994a3e,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
005cc5d858319f13f88228f62341e5a4270f8e75,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409
006d3c79b9ed677280f8ddbc422d7b0fedd6d1fa,8312,1209,2290,1352,26685,20048,6308,16216,5039,7960,...,34535,15185,22648,4590,35471,20634,9682,25299,6835,17409


In [100]:
content_most_rated_test_df = pd.concat([content_most_rated_in_data_df, new_users_recommendations])
print(content_most_rated_test_df.shape)

(1000, 100)


In [101]:
# Save the content-based filtering recommendations 
save_predictions(content_most_rated_test_df, os.path.join("data", "content_recommendations.csv"))

Saved to csv in 'data/content_recommendations.csv'.
